## АвтОбрЕЯ: домашнее задание 2
Елена Зайцева (БКЛ201)

### 1. корпус
- Размер [корпуса](https://docs.google.com/document/d/1EcFc3suDQ-EQi7Ft9J2dthMgPoAm1szYU7UHk9stEV8/edit?usp=sharing)  -- 373 слова.
- В корпусе сложные для автоматического посттеггинга случаи (в основном для них pos-tagger должен хорошо анализировать контекст): 
    + неизвестные корни: кусочек от стихотворения "Бармаглот" Кэрролла с выдуманными словами (тут будет важно учитывать контекст и возможно смотреть на продуктивные аффиксы
    + омонимичные пары (предлоги и наречия -- "вблизи", прилаг. и сущ. -- "больные", прич. и сущ. -- "данные")
    + разные типы числительных, записанных цифрами (различение порядковых (как прил.) и количественных (как числ.) + римские цифры (обычно как прил.))
    + имена собственные, фамилии, которые могут определяться как прил. или что-то другое, (Ленин, Кривошеин (как прил.), Шалтай-Болтай (как глаг.))
    + всякие сокращения вроде "г.", "ок.", "н. э.", "ГэВ" + инициалы перед фамилией

Для анализа взят тегсет [Universal POS tags](https://universaldependencies.org/u/pos/), так как он, кажется, часто используется теггерами (в частности, spacy и stanza), достаточно удобен для русского, не возникает трудности с тем, как размечать причастия, деепричастия, инфинитив (все обозначается как VERB).

Были сделаны некоторые изменения: AUX заменен на VERB, CCONJ и SCONJ объединены в CONJ.

In [91]:
import csv

In [92]:
with open('text-corpus.txt', 'r', encoding='utf-8') as f1:
    text = f1.read()

In [107]:
all_tok_pos = []
with open('POS-corpus.tsv', 'r', newline='', encoding='utf-8') as f:
    words_tsv = csv.reader(f, delimiter='\t')
    for l in words_tsv:
        all_tok_pos.extend(l)

In [111]:
all_pos = []
all_tok = []
for i in range(1, len(all_tok_pos), 2):
    all_pos.append(all_tok_pos[i])
for i in range(0, len(all_tok_pos), 2):
    all_tok.append(all_tok_pos[i])

### 2. три POS-теггера
*stanza, pymorphy, spacy*

#### pymorphy2
(использует собственные [теги](https://pymorphy2.readthedocs.io/en/stable/user/grammemes.html#grammeme-docs))

In [1]:
from pymorphy2 import MorphAnalyzer

In [2]:
morph = MorphAnalyzer()

In [133]:
pos_pymor = []
for i in all_tok:
    pos_pymor.append(str(morph.parse(i.strip('.'))[0].tag).split(',')[0].split(' ')[0])

#### stanza
(использует Universal POS tags)

In [11]:
import stanza

In [12]:
stanza.download('ru')
ppln = stanza.Pipeline('ru', processors='tokenize,pos')

2022-10-11 12:47:38 INFO: Downloading default packages for language: ru (Russian) ...


2022-10-11 12:48:39 INFO: Finished downloading models and saved to C:\Users\espia\stanza_resources.
2022-10-11 12:48:39 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2022-10-11 12:48:41 INFO: Loading these models for language: ru (Russian):
| Processor | Package   |
-------------------------
| tokenize  | syntagrus |
| pos       | syntagrus |

2022-10-11 12:48:41 INFO: Use device: cpu
2022-10-11 12:48:41 INFO: Loading: tokenize
2022-10-11 12:48:41 INFO: Loading: pos
2022-10-11 12:48:42 INFO: Done loading processors!


In [184]:
pos_stanza = []
tok_stanza = []
ana_stanza = ppln(text)
for i in ana_stanza.sentences:
    for w in i.words:
        pos_stanza.append(w.upos)
        tok_stanza.append(w.text)

##### (опасная зона: исправления)
Некоторые слова разделились по дефису или не распознались как сокращения, их нужно вернуть в "нормальную" форму, чтобы потом было можно посчитать accuracy.

In [187]:
# некоторые исправления
tok_stanza[tok_stanza.index('э')] = 'э.'
del pos_stanza[tok_stanza.index('э.')+1]
del tok_stanza[tok_stanza.index('э.')+1]

In [188]:
tok_stanza[tok_stanza.index('электрон')] = 'электрон-позитронный'
del tok_stanza[tok_stanza.index('электрон-позитронный')+1]
del pos_stanza[tok_stanza.index('электрон-позитронный')+1]
del tok_stanza[tok_stanza.index('электрон-позитронный')+1]
del pos_stanza[tok_stanza.index('электрон-позитронный')+1]

In [189]:
tok_stanza[tok_stanza.index('Шалтай')] = 'Шалтай-Болтай'
del tok_stanza[tok_stanza.index('Шалтай-Болтай')+1]
del pos_stanza[tok_stanza.index('Шалтай-Болтай')+1]
del tok_stanza[tok_stanza.index('Шалтай-Болтай')+1]
del pos_stanza[tok_stanza.index('Шалтай-Болтай')+1]

In [197]:
tok_stanza[tok_stanza.index('г')] = 'г.'
del pos_stanza[tok_stanza.index('г.')+1]
del tok_stanza[tok_stanza.index('г.')+1]

#### spacy
(использует Universal POS tags)

In [36]:
import spacy

In [37]:
nlp = spacy.load("ru_core_news_sm")

In [208]:
sentences = text.split('\n')
tok_spacy = []
pos_spacy = []
for sent in sentences:
    ana_spacy = nlp(sent)
    for token in ana_spacy:
        tok_spacy.append(token.text)
        pos_spacy.append(token.pos_)

##### (опасная зона: исправления)
Аналогично ситуации со stanza. (Причем spacy хуже размечает такие случаи)

In [235]:
# некоторые исправления
tok_spacy[tok_spacy.index('э')] = 'э.'
del pos_spacy[tok_spacy.index('э.')+1]
del tok_spacy[tok_spacy.index('э.')+1]

tok_spacy[tok_spacy.index('электрон')] = 'электрон-позитронный'
del tok_spacy[tok_spacy.index('электрон-позитронный')+1]
del pos_spacy[tok_spacy.index('электрон-позитронный')+1]
del tok_spacy[tok_spacy.index('электрон-позитронный')+1]
del pos_spacy[tok_spacy.index('электрон-позитронный')+1]

tok_spacy[tok_spacy.index('Шалтай')] = 'Шалтай-Болтай'
del tok_spacy[tok_spacy.index('Шалтай-Болтай')+1]
del pos_spacy[tok_spacy.index('Шалтай-Болтай')+1]
del tok_spacy[tok_spacy.index('Шалтай-Болтай')+1]
del pos_spacy[tok_spacy.index('Шалтай-Болтай')+1]

tok_spacy[tok_spacy.index('г')] = 'г.'
del pos_spacy[tok_spacy.index('г.')+1]
del tok_spacy[tok_spacy.index('г.')+1]

tok_spacy[tok_spacy.index('н')] = 'н.'
del pos_spacy[tok_spacy.index('н.')+1]
del tok_spacy[tok_spacy.index('н.')+1]

tok_spacy[tok_spacy.index('то')] = 'что-то'  # в корпусе только 2 вхождения "то", и первое -- то, которое здесь нужно
del tok_spacy[tok_spacy.index('что-то')-1]
del pos_spacy[tok_spacy.index('что-то')-1]
del tok_spacy[tok_spacy.index('что-то')-1]
del pos_spacy[tok_spacy.index('что-то')-1]

tok_spacy[tok_spacy.index('турецки')] = 'по-турецки'
del tok_spacy[tok_spacy.index('по-турецки')-1]
del pos_spacy[tok_spacy.index('по-турецки')-1]
del tok_spacy[tok_spacy.index('по-турецки')-1]
del pos_spacy[tok_spacy.index('по-турецки')-1]

In [240]:
tok_spacy[tok_spacy.index('ок')] = 'ок.'
del pos_spacy[tok_spacy.index('ок.')+1]
del tok_spacy[tok_spacy.index('ок.')+1]

### 3. Подсчет accuracy + конвертер
*Спойлер: у stanza accuracy выше всех*

In [256]:
from sklearn.metrics import accuracy_score

In [250]:
def convert_st_sp_tags(pos_lst):
    tags_st_sp = {'AUX': 'VERB', 'CCONJ': 'CONJ', 'PROPN': 'NOUN', 'SCONJ': 'CONJ', 'SYM': 'PUNCT', 'X': 'PUNCT'}
    converted_lst = []
    for tag in pos_lst:
        if tag in tags_st_sp.keys():
            converted_lst.append(tags_st_sp[tag])
        else:
            converted_lst.append(tag)
    return converted_lst

In [251]:
def convert_pymor_tags(pos_lst):
    tags_pymor = {'NOUN': 'NOUN', 'ADJF': 'ADJ', 'ADJS': 'ADJ', 'COMP': 'ADV', 'VERB': 'VERB', 
                  'INFN': 'VERB', 'PRTF': 'VERB', 'PRTS': 'VERB', 'GRND': 'VERB', 'NUMR': 'NUM', 
                  'ADVB': 'ADV', 'NPRO': 'PRON', 'PRED': 'ADV', 'PREP': 'ADP', 'CONJ': 'CONJ', 
                  'PRCL': 'PART', 'INTJ': 'INTJ', 'LATN': 'NOUN', 'PNCT': 'PUNCT', 'NUMB': 'NUM', 
                  'intg': 'NUM', 'real': 'NUM', 'ROMN': 'NUM', 'UNKN': 'PUNCT'}
    converted_lst = []
    for tag in pos_lst:
        converted_lst.append(tags_pymor[tag])
    return converted_lst

In [258]:
# проверка pymorphy
pos__pymor_pred = convert_pymor_tags(pos_pymor)
accuracy_score(all_pos, pos__pymor_pred)

0.8816705336426914

In [259]:
# проверка stanza
pos_st_pred = convert_st_sp_tags(pos_stanza)
accuracy_score(all_pos, pos_st_pred)

0.9582366589327146

In [260]:
# проверка spacy
pos_sp_pred = convert_st_sp_tags(pos_spacy)
accuracy_score(all_pos, pos_sp_pred)

0.9443155452436195

### 4. Про n-граммы

1. биграммы с "не": "не" + прилагательное / наречие / глагол (чтобы различать положительную и отрицательную окраску: случаи типа "рекомендую" vs. "не рекомендую")
2. биграммы прилагательное + существительное (может быть важно в отзывах, потому что, например, "громкий звук" в "громкий звук клавиатуры" скорее будет в отрицательных отзывах, а "громкий звук" в "громкий звук колонок" -- скорее в положительных)
3. много / мало + существительное / глагол (тоже помогает различить настроение: "мало интересного" vs. "много интересного", "мало экшена" vs. "много экшена" (обычно второе чаще как положительное?), "мало платят" vs. "много платят" и т. п.)
    1. тут можно добавить ещё три-грамму с "чересчур" (4-грамму, если соединить с п. 2), которая может менять смысл на противоположный (например, "громкий звук колонок" vs. "чересчур громкий звук колонок")